In [5]:
import bioread
import pandas as pd
import heartpy as hp
import neurokit2 as nk

acq_file = bioread.read_file(r'C:\Users\User\Downloads\80_28.1.24 (NPU_template).acq')

channels_to_extract = ['ECG - ECG100C', 'C5 - Expression' ]
data = {}
for channel_name in channels_to_extract:
    channel = acq_file.named_channels[channel_name]
    data[channel_name] = channel.data

df = pd.DataFrame(data)
print(df)


ImportError: DLL load failed while importing _iterative: The specified module could not be found.

In [23]:
ecg_data =  {
    "n_cue": [],
    "n_no_cue": [],
    "p_cue": [],
    "p_no_cue": [],
    "u_cue": [],
    "u_no_cue": []
}
for index, row in df.iterrows():
    value = row['C5 - Expression']
    ecg_value = row['ECG - ECG100C']
    
    
    if value == 020.0:
        ecg_data["n_cue"].append(ecg_value)
    if value == 030.0:
        ecg_data["n_no_cue"].append(ecg_value)
    if value == 120.0:
        ecg_data["p_cue"].append(ecg_value)
    if value == 130.0:
        ecg_data["p_no_cue"].append(ecg_value)
    if value == 220.0:
        ecg_data["u_cue"].append(ecg_value)
    if value == 230.0:
        ecg_data["u_no_cue"].append(ecg_value)
        
max_length = max(len(ecg_data[key]) for key in ecg_data)
for key in ecg_data:
    while len(ecg_data[key]) < max_length:
        ecg_data[key].append(None)
df_ecg = pd.DataFrame(ecg_data)
print(df_ecg)


    
         


         n_cue  n_no_cue     p_cue  p_no_cue     u_cue  u_no_cue
0    -0.092621 -0.018768 -0.009613 -0.011139  0.035706 -0.032196
1    -0.091858 -0.019073 -0.009308 -0.011597  0.036163 -0.031586
2    -0.091248 -0.018921 -0.009003 -0.011902  0.036774 -0.031433
3    -0.090790 -0.019073 -0.009460 -0.012665  0.037384 -0.031281
4    -0.089874 -0.018768 -0.009460 -0.012970  0.037689 -0.031128
...        ...       ...       ...       ...       ...       ...
1194       NaN  0.011444       NaN       NaN       NaN       NaN
1195       NaN  0.011902       NaN       NaN       NaN       NaN
1196       NaN  0.012360       NaN       NaN       NaN       NaN
1197       NaN  0.012817       NaN       NaN       NaN       NaN
1198       NaN  0.013428       NaN       NaN       NaN       NaN

[1199 rows x 6 columns]
0   -0.092621
1   -0.091858
2   -0.091248
3   -0.090790
4   -0.089874
Name: n_cue, dtype: float64


In [28]:
## trying with HEARTPY
ecg_sampling_rate = 2000
mean_heart_rate = {}
for column in df_ecg.columns:
    working_data, measures = hp.process(df_ecg[column].values, ecg_sampling_rate)
    mean_heart_rate[column] = measures['bpm']
    mean_heart_rate = {column: sum(heart_rates) / len(heart_rates) for column, heart_rates in mean_heart_rate.items()}
print("Mean Heart Rates:")
for column, mean_hr in mean_heart_rate.items():
    print(f"{column}: {mean_hr} bpm")

## trying with neurokit   
ecg_sampling_rate = 2000
heart_rate_data = {}
for column in df_ecg.columns:
    signals = nk.ecg_process(df_ecg[column], ecg_sampling_rate)
    heart_rate = nk.ecg_rate(signals)
    heart_rate_data[column] = heart_rate
print("Heart Rates:")
for column, heart_rate in heart_rate_data.items():
    print(f"{column}: {heart_rate.mean()} bpm")

heart_rate_df = pd.DataFrame(heart_rate_data)

mean_heart_rate_per_column = heart_rate_df.mean(axis=0)

print("Mean Heart Rate per column:")
print(mean_heart_rate_per_column)
    
    







    

ValueError: negative dimensions are not allowed